
## Lab: Ngrams and Text Clustering

### Introduction

In this Notebook, we will explore the concept of n-grams, which are combinations of one or more tokens. For example, bigrams are combinations of two tokens, while trigrams are combinations of three tokens. We will peek into the application of clustering and feature selection to text data at the end.

-----

In [ ]:
% matplotlib inline

# Standard imports
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import pprint
from sklearn.datasets import load_files
from sklearn.cross_validation import train_test_split

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import metrics

import string
import nltk
from nltk.stem.porter import PorterStemmer



### n-grams

A [_n-gram_](https://en.wikipedia.org/wiki/N-gram) is a contiguous sequence of **n** items from a parent sequence of items, such as characters or words in a text document. In general, the focus will be solely on words in a document. Thus, until now we have looking at unigrams or single words in a document when building a classification model. But sometimes a combination of words can be more descriptive, for example, _unbelievably bad_ is in general a more powerful description than just _bad_. This resulted in the concept of _n-gram_, where collections of words can be treated as features.

While this clearly can improve classification power, it also increases computational requirements. This is a result of the exponential rise in the number of possible features. For example, given $n$ words, we have $n \times (n - 1)$ possible bigrams, and so on for higher order combinations. While this is not a problem for small vocabularies, for larger vocabularies (and corresponding documents) the number of possible features can quickly become very large. Thus, many text mining applications will make use of Hadoop or Spark clusters to leverage the inherent parallelism in these tasks.

Below code cell will illustrates using n-grams, using a single sentence.

-----

In [ ]:
text = 'Information retrieval is different from Data mining'

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(ngram_range=(1,3))

tk_func = cv.build_analyzer()

import pprint
pp = pprint.PrettyPrinter(indent=2, depth=1, width=80, compact=True)

pp.pprint(tk_func(text))

In [ ]:
in_list = []
in_list.append(text)

cv = cv.fit(in_list)

In [ ]:
cv.vocabulary_.items()

In [ ]:
import operator
my_voc = sorted(cv.vocabulary_.items(), key=operator.itemgetter(1))

print('Token mapping:')
print(40*'-')

for tokens, rank in my_voc:
    print(rank, tokens)

### N-gram Classification

Having n-grams offers improved classification, since word or token combinations often include more information than single words or tokens. For example, _University Illinois_ means more than just _University_ and _Illinois_. We can build on our previous simple text classification pipeline to now develop a more complete code example that builds a feature vector containing both single words and n-grams from the documents. We use this new sparse matrix to classify the documents by using our simple Naive Bayes classifier, which obtains slightly better results. First, we load the movie review data.

-----

In [ ]:
import nltk
mvr = nltk.corpus.movie_reviews

from sklearn.datasets import load_files

data_dir = '../../../datasets/movie_reviews'


#data_dir = '/home/data_scientist/nltk_data/corpora/movie_reviews'
mvr = load_files(data_dir, shuffle = False)
print('Number of Reviews: {0}'.format(len(mvr.data)))

from sklearn.model_selection import train_test_split

mvr_train, mvr_test, y_train, y_test = train_test_split(
    mvr.data, mvr.target, test_size=0.25, random_state=23)
print('Number of Reviews in train: {0}'.format(len(mvr_train)))
print('Number of Reviews in test: {0}'.format(len(mvr_test)))

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import metrics

tools = [('cv', CountVectorizer()), ('nb', MultinomialNB())]
pclf = Pipeline(tools)


# Lowercase and restrict ourselves to about half the available features
pclf.set_params(cv__stop_words = 'english', \
                cv__ngram_range=(1,2), \
                cv__lowercase=True)

pclf.fit(mvr_train, y_train)
y_pred = pclf.predict(mvr_test)
print(metrics.classification_report(y_test, y_pred, target_names = mvr.target_names))

In [ ]:
# Extract the classifier
clf = pclf.steps[1][1]
print('Number of Features = {}'.format(clf.feature_log_prob_.shape[1]))

-----

Now we can repeat the results, but now use unigrams, bigrams, and trigrams. Since this will produce a document term matrix that likely exceeds our computational resources, we impose two cuts on the tokens included in our DTM. First, we impose a minimum feature term that requires a term to be present in at least two documents. Second, we set a maximum frequency of 50%, such that any term occurring in more than fifty percent of all documents will be ignored (these are likely all stop words, but this value can be reduced to a lower value to trim frequently occurring words that add little to the accuracy. Notice that our overall number of features is much smaller, even though we are also using trigrams in this classification pipeline.

-----

In [ ]:
pclf.set_params(cv__stop_words = 'english', \
                cv__ngram_range=(1,3), \
                cv__lowercase=True, \
                cv__min_df=2, \
                cv__max_df=0.5)

pclf.fit(mvr_train, y_train)
y_pred = pclf.predict(mvr_test)
print(metrics.classification_report(y_test, y_pred, target_names = mvr.target_names))

In [ ]:
# Extract the classifier
clf = pclf.steps[1][1]
print('Number of Features = {}'.format(clf.feature_log_prob_.shape[1]))

### Clustering Analysis

We can also apply clustering analysis to our feature matrix. While finding an unknown number of clusters in text documents can be difficult, we can learn about our data by identifying the clusters for our **known** labels. To demonstrate, in the following code cells, we employ k-means to find two clusters in our feature matrix (the movie reviews), after which we identify the most frequently used words in each cluster.

-----

In [ ]:
from sklearn.cluster import KMeans

true_k = 2

km = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)

from sklearn.feature_extraction.text import CountVectorizer

# Verify attributes

cv = CountVectorizer(stop_words = 'english', \
                     ngram_range=(1, 3), max_features=100000)

train_counts = cv.fit_transform(mvr_train)
test_data = cv.fit_transform(mvr_test)

km.fit(test_data)

In [ ]:
top_tokens = 20
labels = ['Neg', 'Pos']

print('Top {} tokens per cluster:\n'.format(top_tokens))

order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = cv.get_feature_names()

for idx in range(true_k):
    print("Cluster {0}:".format(idx), end='')
    for jdx in order_centroids[idx, :top_tokens]:
        print(' {0}'.format(terms[jdx]), end='')
    print('\n')

### Dimension Reduction

The document term matrices can become quite large. We tried to reduce the number of features by using stop words, by using a consistent case, and by performing stemming. On the other hand, we have enabled exponential increases in the feature space by including n-grams but at the same time we are once again restricting the feature space by using the `max_features` or the `max_df` and `min_df` attributes. 

The traditional dimensional reduction technique we have employed in the past is PCA. However, PCA can be difficult to use with text data given the large sizes of the matrices (since a matrix inversion can be required). Alternative techniques, such as incremental PCA or Truncated SVD could be used. 

In the case of document term matrices, we are less interested in finding a reduced dimensional space than we already have to remove features that contain little or no information. Here, the problem of dimension reduction becomes one of optimal feature selection. scikit learn's `SelectKBest` method can be used to identify the best $k$ features. 

In the following code cells, we will create a vectorizer first, train it on data to demonstrate the accuracy and the number of features required to achieve that accuracy. Later, we employ `SelectKBest` to identify the best number of features, where the number is predetermined, to achieve a similar accuracy. Finally we will try to demonstrate that less than ten percent of the original features are required to achieve the same level of accuracy.

We will switch back to using the newsgroups dataset.

-----

In [ ]:
# Split into training and testing
from sklearn.datasets import fetch_20newsgroups

train = fetch_20newsgroups(data_home='/dsa/data/DSA-8630/newsgroups/', subset='train', shuffle=True, random_state=23)
test = fetch_20newsgroups(data_home='/dsa/data/DSA-8630/newsgroups/', subset='test', shuffle=True, random_state=23)

In [ ]:
# Following Example was insipred by scikit learn demo
# http://scikit-learn.org/stable/auto_examples/text/document_classification_20newsgroups.html

from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(sublinear_tf=True, max_df=0.5, stop_words='english')

In [ ]:
# First, train on normal set of features, baseline performance.

train_counts = tf.fit_transform(train['data'])
test_data = tf.transform(test['data'])

nb = MultinomialNB()
nb = nb.fit(train_counts, train['target'])
predicted = nb.predict(test_data)

print("Prediction accuracy = {0:5.1f}%".format(100.0 * nb.score(test_data, test['target'])))
print('Number of Features = {}'.format(nb.feature_log_prob_.shape[1]))

In [ ]:
# Now employ feature selection
from sklearn.feature_selection import SelectKBest, chi2

# Number of features to keep
num_k = 10000

# Employ select k best wiht chi2 since this works with sparse matrices.
ch2 = SelectKBest(chi2, k=num_k)
xtr = ch2.fit_transform(train_counts, train['target'])
xt = ch2.transform(test_data)

In [ ]:
# Train simple model and compute accuracy
nb = nb.fit(xtr, train['target'])
predicted = nb.predict(xt)

print("NB prediction accuracy = {0:5.1f}%".format(100.0 * nb.score(xt, test['target'])))
print('Number of Features = {}'.format(nb.feature_log_prob_.shape[1]))

-----

We can use the feature selection to identify the top features for each category. First we extract the feature names, after which we extract the importance (or support) for each feature. By sorting the array containing the important features, we can identify the top tokens.

-----

In [ ]:
feature_names = tf.get_feature_names()

indices = ch2.get_support(indices=True)
feature_names = np.array([feature_names[idx] for idx in indices])

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=2, depth=1, width=80, compact=True)

top_count = 5

for idx, target in enumerate(train['target_names']):
    top_names = np.argsort(nb.coef_[idx])[-top_count:]
    tn_lst = [name for name in feature_names[top_names]]
    tn_lst.reverse()

    print('\n{0}:'.format(target))
    pp.pprint(tn_lst)